<a href="https://colab.research.google.com/github/benjaminnigjeh/automated_data_mining/blob/main/automated_datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Fisher-py module
This is a module for parsing Thermo raw files. Since it's run time is pythonnet, it requires installation of .NET framework.

In [ ]:
!apt-get update
!apt-get install -y mono-complete
!pip install fisher-py

#Import external libraries

In [45]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
import re
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

#Helper Functions

In [46]:
def helper_regex(text):
    match = re.search(rf"{'Full'}\s+(\w+)", text)
    if match:
        return match.group(1)
    return None

def cast_func_simple(scan_number):
    data_intensities = [0]*1600
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities

    for j in range(0,len(scan_masses)):
        index = round(scan_masses[j])
        if index > 400 and index < 2000:
            data_intensities[index-400] = scan_intensities[j] + data_intensities[index-400]

    return(list(data_intensities))

def casting(file_path, cast_path):
    raw = RawFile(file_path)

    cast_spectra = []
    mz_value = []
    tic = []
    scan_number = []
    retention_time = []
    sample_name = []

    for i in tqdm(range(1, raw.number_of_scans), desc="Processing scans", ncols=100):
        raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
        if str(helper_regex(raw_scan.scan_type)) == 'ms2':
            cast_spectra.append(cast_func_simple(i))
            mz_value.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
            tic.append(raw_scan.scan_statistics.tic)
            scan_number.append(raw_scan.scan_statistics.scan_number)
            retention_time.append(raw.get_retention_time_from_scan_number(raw_scan.scan_statistics.scan_number))
            sample_name = 'None'
    print("\nScan processing complete!")
    scan_dict = {'scan': scan_number, 'retntion time': retention_time, 'm/z': mz_value, 'TIC': tic, 'cast spectra': cast_spectra, 'sample name' : sample_name}

    with open(cast_path, 'wb') as f:
        pickled_file = pickle.dump(scan_dict, f)
    return(pickled_file)

#procedure

In [43]:
file_path = '/content/drive/MyDrive/20231221_msz1718_MAbPac-02_Neo_ExtraValveT_ACN30S1_TreatmentA_biorep01_techrep01.raw'
cast_path = '/content/cast'
casting(file_path, cast_path)

Processing scans: 100%|█████████████████████████████████████████| 3269/3269 [01:04<00:00, 50.93it/s]



Scan processing complete!


In [44]:
with open('/content/cast', 'rb') as f:
    data = pickle.load(f)

pd.DataFrame(data)


,scan,retntion time,m/z,TIC,cast spectra,sample name
0,3,0.020602,532.6573,44326.285156,"[0, 0, 92.89836120605469, 174.21189880371094, ...",None
1,32,0.284152,533.2592,48184.800781,"[0, 0, 0, 0, 105.70592498779297, 0, 0, 0, 0, 0...",None
2,145,1.285471,532.8577,43661.265625,"[0, 0, 104.3642807006836, 0, 0, 0, 0, 0, 0, 0,...",None
3,259,2.295235,532.8579,48168.136719,"[0, 124.89591217041016, 0, 109.54644775390625,...",None
4,372,3.296624,532.6570,54236.125000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 612.34...",None
...,...,...,...,...,...,...
1774,3151,28.889778,739.8889,58567.109375,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None
1775,3157,28.951688,1687.7987,1335.449219,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None
1776,3205,29.386061,1816.3376,1464.566406,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None
1777,3241,29.713241,830.5879,7015.328125,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None
